# RESUMEN SESIÓN 2: Inyección de Dependencias y Arquitectura Modular

**Cheat-sheet ejecutable con snippets listos para usar**

## IMPORTS NECESARIOS

In [ ]:
from fastapi import FastAPI, Depends, HTTPException, Header, APIRouter, Query
from fastapi.testclient import TestClient
from typing import Annotated, Generator, List
import time

---

## 1️⃣ ANNOTATED: Tipos Reutilizables

### Tabla Comparativa

| Enfoque | Código | Ventajas |
|---------|--------|----------|
| **Sin Annotated** | `def endpoint(user: str = Depends(get_user)):` | Simple para 1 uso |
| **Con Annotated** | `UserDep = Annotated[str, Depends(get_user)]`<br>`def endpoint(user: UserDep):` | Reutilizable, limpio |

In [ ]:
# SNIPPET: Patrón Annotated

# 1. Definir la dependencia
def obtener_usuario(user_id: str = Header()) -> str:
    return user_id

# 2. Crear tipo reutilizable
UsuarioDep = Annotated[str, Depends(obtener_usuario)]

# 3. Usar en múltiples endpoints
app = FastAPI()

@app.get("/perfil")
def ver_perfil(usuario: UsuarioDep):
    return {"usuario": usuario}

@app.get("/configuracion")
def ver_config(usuario: UsuarioDep):
    return {"usuario": usuario, "config": {}}

---

## 2️⃣ DEPENDS: Inyección de Dependencias

### Tipos de Dependencias

| Tipo | Cuándo usar | Ejemplo |
|------|-------------|----------|
| **Función simple** | Valores estáticos, configuración | `def get_config(): return {...}` |
| **Función con params** | Leer headers, query params | `def get_user(token: str = Header()): ...` |
| **Clase** | Lógica de negocio compleja | `class UserService: ...` |
| **Con yield** | Recursos (BD, archivos) | `def get_db(): db = DB(); yield db; db.close()` |

In [ ]:
# SNIPPET: Dependencia que lee Header

def verificar_api_key(x_api_key: str = Header()) -> str:
    """Valida API key desde header."""
    if x_api_key != "clave-secreta":
        raise HTTPException(status_code=403, detail="API key inválida")
    return x_api_key

ApiKeyDep = Annotated[str, Depends(verificar_api_key)]

app = FastAPI()

@app.get("/datos")
def obtener_datos(api_key: ApiKeyDep):
    return {"datos": "confidenciales", "key": api_key}

---

## 3️⃣ YIELD: Gestión de Recursos

### Flujo de Ejecución

```
┌─────────────────────────────────┐
│ 1. Setup (conectar)             │ ← Antes del endpoint
├─────────────────────────────────┤
│ 2. yield recurso                │ ← Entrega al endpoint
├─────────────────────────────────┤
│ 3. Endpoint usa recurso         │
├─────────────────────────────────┤
│ 4. Teardown (desconectar)       │ ← Después del endpoint (SIEMPRE)
└─────────────────────────────────┘
```

In [ ]:
# SNIPPET: Dependencia con yield para BD

class Database:
    def __init__(self):
        self.connected = False
    
    def connect(self):
        print("📦 Conectando...")
        self.connected = True
    
    def disconnect(self):
        print("📦 Desconectando...")
        self.connected = False
    
    def query(self, sql: str):
        return ["datos", "de", "bd"]

def obtener_db() -> Generator[Database, None, None]:
    """Gestiona ciclo de vida de la BD."""
    db = Database()
    db.connect()  # Setup
    try:
        yield db  # Entrega
    finally:
        db.disconnect()  # Teardown (SIEMPRE se ejecuta)

DatabaseDep = Annotated[Database, Depends(obtener_db)]

app = FastAPI()

@app.get("/consulta")
def hacer_consulta(db: DatabaseDep):
    return {"resultado": db.query("SELECT *")}

---

## 4️⃣ SUB-DEPENDENCIAS

### Composición de Lógica

| Nivel | Dependencia | Descripción |
|-------|-------------|-------------|
| 1 | `get_timestamp()` | Genera timestamp |
| 2 | `get_logger(timestamp)` | Usa timestamp para crear logger |
| 3 | `endpoint(logger)` | Usa logger con timestamp automático |

In [ ]:
# SNIPPET: Sub-dependencias

# Dependencia nivel 1
def get_timestamp() -> float:
    return time.time()

TimestampDep = Annotated[float, Depends(get_timestamp)]

# Dependencia nivel 2 (usa nivel 1)
def get_logger(ts: TimestampDep) -> dict:
    return {"timestamp": ts, "nivel": "INFO"}

LoggerDep = Annotated[dict, Depends(get_logger)]

# Endpoint usa nivel 2 (que usa nivel 1 automáticamente)
app = FastAPI()

@app.get("/log")
def crear_log(logger: LoggerDep):
    return {"log": logger}

---

## 5️⃣ APIROUTER: Modularización

### Configuración del Router

| Parámetro | Tipo | Descripción | Ejemplo |
|-----------|------|-------------|---------|
| `prefix` | str | Prefijo común | `/api/v1` |
| `tags` | List[str] | Etiquetas Swagger | `["Productos"]` |
| `dependencies` | List[Depends] | Aplica a todos | `[Depends(verify_admin)]` |

In [ ]:
# SNIPPET: Router completo

router_productos = APIRouter(
    prefix="/api/v1/productos",
    tags=["Productos"]
)

@router_productos.get("/")
def listar_productos():
    return {"productos": ["Laptop", "Mouse"]}

@router_productos.post("/")
def crear_producto(nombre: str):
    return {"id": 1, "nombre": nombre}

@router_productos.delete("/{id}")
def eliminar_producto(id: int):
    return {"mensaje": f"Producto {id} eliminado"}

# Registrar en app
app = FastAPI()
app.include_router(router_productos)

---

## 6️⃣ DEPENDENCIAS GLOBALES

### Aplicación Automática

```
Router con dependencies=[Depends(auth)]
    │
    ├─ GET /admin/usuarios      ← Auth automático
    ├─ GET /admin/config        ← Auth automático  
    └─ DELETE /admin/usuarios   ← Auth automático
```

In [ ]:
# SNIPPET: Dependencias globales en router

def verificar_admin(x_role: str = Header()) -> bool:
    """Valida que el usuario sea admin."""
    if x_role != "admin":
        raise HTTPException(status_code=403, detail="Requiere rol admin")
    return True

# Router con dependencia global
router_admin = APIRouter(
    prefix="/admin",
    tags=["Admin"],
    dependencies=[Depends(verificar_admin)]  # ← Aplica a TODOS
)

@router_admin.get("/usuarios")
def listar_usuarios():
    return {"usuarios": ["user1", "user2"]}

@router_admin.get("/config")
def ver_config():
    return {"debug": False}

app = FastAPI()
app.include_router(router_admin)

---

## 7️⃣ SERVICIOS: Service Layer Pattern

### Problema de Persistencia

| Enfoque | Código | Resultado |
|---------|--------|----------|
| **❌ MAL** | `self.data = []` en `__init__` | Se pierde entre requests |
| **✅ BIEN** | `self.data = GLOBAL_DB` | Persiste entre requests |

In [ ]:
# SNIPPET: Servicio con persistencia

# Variable global que simula BD
TAREAS_DB = []
NEXT_ID = {"valor": 1}

class TareaService:
    """Servicio para gestionar tareas."""
    
    def __init__(self):
        # Referencias a globales (NO copias)
        self.tareas = TAREAS_DB
        self.next_id = NEXT_ID
    
    def crear(self, titulo: str) -> dict:
        tarea = {
            "id": self.next_id["valor"],
            "titulo": titulo,
            "completada": False
        }
        self.tareas.append(tarea)
        self.next_id["valor"] += 1
        return tarea
    
    def listar(self) -> List[dict]:
        return self.tareas

def get_service() -> TareaService:
    return TareaService()

ServiceDep = Annotated[TareaService, Depends(get_service)]

app = FastAPI()

@app.post("/tareas")
def crear(titulo: str, service: ServiceDep):
    return service.crear(titulo)

@app.get("/tareas")
def listar(service: ServiceDep):
    return service.listar()

---

## 8️⃣ ARQUITECTURA COMPLETA: 3 Capas

### Diagrama de Capas

```
┌─────────────────────────────────────┐
│  ROUTER (Presentación)              │  ← Endpoints HTTP
│  - Validación de entrada            │
│  - Serialización de salida          │
└─────────────┬───────────────────────┘
              │ Depends(get_service)
┌─────────────▼───────────────────────┐
│  SERVICE (Lógica de Negocio)        │  ← Reglas de negocio
│  - Validaciones                     │
│  - Transformaciones                 │
└─────────────┬───────────────────────┘
              │ usa Database
┌─────────────▼───────────────────────┐
│  DATABASE (Persistencia)            │  ← SQL/MongoDB
│  - Conexión                         │
│  - Queries                          │
└─────────────────────────────────────┘
```

In [ ]:
# SNIPPET: Arquitectura completa de 3 capas

# Variable global simula BD
DB_PRODUCTOS = [{"id": 1, "nombre": "Laptop", "precio": 1000}]

# 1. CAPA DE DATOS
class Database:
    def __init__(self):
        self.connected = False
        self.productos = DB_PRODUCTOS
    
    def connect(self):
        print("📦 Conectando BD...")
        self.connected = True
    
    def disconnect(self):
        print("📦 Desconectando BD...")
        self.connected = False
    
    def query(self, sql: str):
        return self.productos

def get_db() -> Generator[Database, None, None]:
    db = Database()
    db.connect()
    try:
        yield db
    finally:
        db.disconnect()

DatabaseDep = Annotated[Database, Depends(get_db)]

# 2. CAPA DE SERVICIO
class ProductoService:
    def __init__(self, db: Database):
        self.db = db
    
    def listar(self):
        return self.db.query("SELECT * FROM productos")
    
    def crear(self, nombre: str, precio: float):
        nuevo_id = max(p["id"] for p in self.db.productos) + 1
        producto = {"id": nuevo_id, "nombre": nombre, "precio": precio}
        self.db.productos.append(producto)
        return producto

def get_service(db: DatabaseDep) -> ProductoService:
    return ProductoService(db)

ServiceDep = Annotated[ProductoService, Depends(get_service)]

# 3. CAPA DE PRESENTACIÓN
router = APIRouter(prefix="/api/productos", tags=["Productos"])

@router.get("/")
def listar(service: ServiceDep):
    return service.listar()

@router.post("/")
def crear(nombre: str, precio: float, service: ServiceDep):
    return service.crear(nombre, precio)

app = FastAPI()
app.include_router(router)

---

## 9️⃣ HTTPEXCEPTION: Manejo de Errores

### Códigos HTTP Comunes

| Código | Significado | Cuándo usar |
|--------|-------------|-------------|
| `400` | Bad Request | Datos de entrada inválidos |
| `401` | Unauthorized | No está autenticado |
| `403` | Forbidden | Autenticado pero sin permisos |
| `404` | Not Found | Recurso no existe |
| `500` | Internal Error | Error del servidor |

In [ ]:
# SNIPPET: Lanzar excepciones HTTP

def validar_token(authorization: str = Header()) -> str:
    """Valida token de autorización."""
    if not authorization.startswith("Bearer "):
        raise HTTPException(
            status_code=401,
            detail="Token no proporcionado"
        )
    
    token = authorization.replace("Bearer ", "")
    
    if token != "token-valido":
        raise HTTPException(
            status_code=403,
            detail="Token inválido"
        )
    
    return token

TokenDep = Annotated[str, Depends(validar_token)]

app = FastAPI()

@app.get("/privado")
def datos_privados(token: TokenDep):
    return {"datos": "confidenciales", "token": token}

---

## 🔟 PATRONES ÚTILES

### Header Authentication

In [ ]:
# SNIPPET: Autenticación con header

def get_current_user(authorization: str = Header()) -> str:
    """Extrae usuario del header Authorization."""
    token = authorization.replace("Bearer ", "")
    # Aquí validarías JWT, etc.
    return "user_id_123"

CurrentUserDep = Annotated[str, Depends(get_current_user)]

### Paginación con Query Parameters

In [ ]:
# SNIPPET: Paginación reutilizable

def get_pagination(
    page: int = Query(1, ge=1),
    size: int = Query(10, ge=1, le=100)
) -> dict:
    """Calcula skip y limit para paginación."""
    return {
        "skip": (page - 1) * size,
        "limit": size
    }

PaginationDep = Annotated[dict, Depends(get_pagination)]

@app.get("/items")
def listar_items(pagination: PaginationDep):
    # items = db.query().skip(pagination['skip']).limit(pagination['limit'])
    return {"pagination": pagination}

### Testing con Override de Dependencias

In [ ]:
# SNIPPET: Override para testing

# Dependencia real
def get_real_db():
    return "Real Database"

app = FastAPI()

@app.get("/data")
def get_data(db = Depends(get_real_db)):
    return {"db": db}

# En tests: reemplazar dependencia
def get_fake_db():
    return "Fake Database for Testing"

app.dependency_overrides[get_real_db] = get_fake_db

client = TestClient(app)
response = client.get("/data")
print(response.json())  # {"db": "Fake Database for Testing"}

---

## 📋 CHECKLIST DE BUENAS PRÁCTICAS

### ✅ DO

- Usar `Annotated` para tipos reutilizables
- Usar `yield` para gestión de recursos (BD, archivos)
- Servicios para lógica de negocio
- Variables globales para simular BD en desarrollo
- Dependencias globales para autenticación en routers
- Sub-dependencias para composición de lógica

### ❌ DON'T

- Repetir `Depends()` en cada parámetro sin `Annotated`
- Lógica de negocio directamente en endpoints
- `self.data = []` sin persistencia (se pierde entre requests)
- Olvidar `finally` en dependencias con `yield`
- Asumir que las instancias persisten entre requests

---

## 🎯 RESUMEN DE CONCEPTOS

| Concepto | Keyword | Propósito |
|----------|---------|----------|
| Inyección de Dependencias | `Depends()` | Proveer valores a endpoints |
| Tipos Reutilizables | `Annotated` | Evitar repetición |
| Gestión de Recursos | `yield` | Setup/teardown automático |
| Composición | Sub-dependencias | Cadenas de lógica |
| Modularización | `APIRouter` | Organizar endpoints |
| Validación Global | `dependencies=[...]` | Proteger routers |
| Capa de Negocio | Servicios | Separar lógica |
| Persistencia | Variables globales | Simular BD |

---

## 🚀 COMANDOS ÚTILES

In [ ]:
# Ejecutar servidor con auto-reload
# uvicorn main:app --reload

# Ver documentación interactiva
# http://127.0.0.1:8000/docs

# Testing
# pytest test_main.py -v

# Ver todos los endpoints
# http://127.0.0.1:8000/openapi.json